Sau khi thực nghiệm và thử nghiệm trên các lượt chạy từng prompt một trên google colab và chạy hàng loạt trên kaggle, đây là những prompt tốt nhất đã được rút ra để chạy Microsoft-Phi2

Để so sánh cụ thể, các prompt này chỉ mang tính chất tương đối trong số lượt chạy của người thực hiện prompt

In [30]:
torch.cuda.empty_cache()

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os
torch.cuda.empty_cache()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
model_id = "microsoft/phi-2"
model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True,
            device_map=None
        ).eval().to('cuda')
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")

inputs = tokenizer("What is 12 * 9?", return_tensors="pt").to('cuda')
outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

2025-06-06 05:23:10.251843: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749187390.440921      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749187390.495208      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is 12 * 9?
<|question_end|>Answer: 108



In [2]:
from collections import Counter
import re
def extract_labels(text):
        matches = re.findall(r'\b(positive|negative|neutral)\b', text.lower())
        return list(set(matches))
def clean_label(text):
        match = re.search(r'\b(positive|negative|neutral)\b', text.lower())
        return match.group(1) if match else 'unknown'
def extract_answer(text):
        matches = re.findall(r'= *\$?(-?\d+(?:\.\d+)?)', text)
        return matches[-1] if matches else None
def extract_target(text):
        for line in reversed(text.splitlines()):
            if line.strip().startswith("####"):
                return line.strip().replace("####", "").strip()
        return None
def split_cot_and_answer(self, text):
        lines = text.strip().splitlines()
        cot_lines = [line for line in lines if not line.strip().startswith("####")]
        answer = extract_target(text)
        return "\n".join(cot_lines), answer
def generate_output(prompt):
        response = model.chat(messages=[
            {"role": "user", "content": prompt}
        ])
        return response["choices"][0]["message"]["content"]
def self_consistency(prompt, num_samples=5):
        outputs = []
        for _ in range(num_samples):
            outputs.append(generate_output(prompt))
        return outputs
def majority_vote(outputs):
        counts = Counter(outputs)
        return counts.most_common(1)[0][0], counts
def cosine_similarity_score(pred, target):
        emb1 = comp_model.encode(pred, convert_to_tensor=True)
        emb2 = comp_model.encode(target, convert_to_tensor=True)
        return float(util.cos_sim(emb1, emb2)[0][0])
def find_top_k_tasks(text, k=3):
        q = vectorizer.transform([text])
        scores = (q @ X.T).flatten()
        top = scores.argsort()[::-1][:k]
        return [task_lib[i] for i in top]
def expand_thoughts(prompt, n=3):
        thoughts = []
        for _ in range(n):
            thoughts.append(generate_output(prompt))
        return thoughts

In [1]:
import json
with open('/kaggle/input/classification/classification_instruction_data_en.jsonl', 'r', encoding='utf-8') as f:
    data = [json.loads(line) for line in f]

In [4]:
data[0]

{'instruction': "Classify the following text into 'positive', 'neutral', or 'negative'.",
 'input': 'between 1900 and 1920 where did most of the migrants to the united states come from',
 'output': 'neutral'}

In [21]:
def generate_output(type, input, max_len=200):
        if type=='generation':
            output = model.generate(
                **input,
                max_new_tokens= max_len,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                num_return_sequences=1,
            )
        else:
            output = model.generate(
                **input,
                max_new_tokens= max_len,
                num_beams=5,
                no_repeat_ngram_size=2,
                early_stopping=True,
                pad_token_id = tokenizer.eos_token_id
            )
        return output

In [6]:
import re

def extract_answer(text):
    matches = re.findall(r'= *\$?(-?\d+(?:\.\d+)?)', text)
    if matches:
        return matches[-1]
    return None
def extract_target(text):
  for line in text.splitlines()[::-1]:
    if line.strip().startswith("####"):
      return line.strip().replace("####","").strip()
  return None

In [2]:
true_labels = []
for i in range(100):
    true_labels.append(data[i]['output'])

In [3]:
from collections import Counter

print(Counter(true_labels))

Counter({'neutral': 82, 'positive': 16, 'negative': 2})


In [10]:
import re
def clean_label(text):
    text = text.lower().strip()
    for label in ["positive", "neutral", "negative"]:
        if label in text:
            return label
    return "UNKNOWN"


In [68]:
def zero_shot_direct(text):
    return f"""Instruction: Return only one word — the sentiment (neutral, positive, or negative).
            Sentence: {text}
            Answer:"""


In [69]:
acc = 0
true_results = []
results = []

for i in range(100):
    prompt = zero_shot_direct(data[i]['input'])

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    input_len = inputs["input_ids"].shape[1]  # Chiều dài prompt token
    outputs = generate_output(type=None, input = inputs, max_len = 30)

    # Lấy phần output chỉ sau prompt
    generated_ids = outputs[0][input_len:]
    de = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
    label = clean_label(de.lower())

    if label == data[i]['output']:
        acc += 1
        true_results.append(label)

    print("Generated:", de)
    print("Target:", data[i]['output'])
    print("Output:", label)

    results.append(label)


Generated: The type of sentiment expressed in the sentence is neutral.
Target: neutral
Output: neutral
Generated: Positive
Target: neutral
Output: positive
Generated: Positive
Target: positive
Output: positive
Generated: Neutral
Target: neutral
Output: neutral
Generated: Ray.
Target: neutral
Output: unknown
Generated: Neutral
Target: positive
Output: neutral
Generated: Neutral
Target: neutral
Output: neutral
Generated: Neutral
Input: 
Output: The sentiment of the sentence is neutral.
Target: neutral
Output: neutral
Generated: The sentiment of the sentence is neutral.
Target: neutral
Output: neutral
Generated: Positive
Target: neutral
Output: positive
Generated: Neutral
Target: neutral
Output: neutral
Generated: The sentiment of the sentence is neutral. The sentence does not express any emotion or attitude towards the subject or the action. It is a factual statement that
Target: neutral
Output: neutral
Generated: The sentence has a neutral sentiment.
Target: neutral
Output: neutral
Gene

In [70]:
print(acc/100)
print(Counter(true_results))
print(Counter(results))

0.66
Counter({'neutral': 59, 'positive': 7})
Counter({'neutral': 69, 'positive': 18, 'unknown': 11, 'negative': 2})


In [58]:
def zero_shot_CoT(text):
    return f"""### Instruction:
               Classify the sentiment of the following sentence into one of the following categories: neutral, positive, or negative.
            
               ### Sentence:
               {text}
            
               ### Response:
               Let's think step by step. Sentiment:"""


In [60]:
acc = 0
true_results = []
results = []

for i in range(100):
    prompt = zero_shot_CoT(data[i]['input'])
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    input_len = inputs["input_ids"].shape[1]  
    outputs = generate_output(type=None, input = inputs, max_len=30)

    generated_ids = outputs[0][input_len:]
    de = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
    label = clean_label(de.lower())

    if label == data[i]['output']:
        acc += 1
        true_results.append(label)

    print("Generated:", de)
    print("Target:", data[i]['output'])
    print("Output:", label)

    results.append(label)


Generated: Neutral. Explanation: The sentence does not express any strong emotions or opinions. It simply states a fact about the migration patterns during a specific time period
Target: neutral
Output: neutral
Generated: Neutral. To determine what is irrelevant to sentiment, we need to consider these lists and judge their relevance one by one: (1) The meaning of
Target: neutral
Output: neutral
Generated: Neutral. Explanation: The sentence does not express any strong emotions or opinions, so it can be classified as neutral. To determine what is irrelevant to
Target: positive
Output: neutral
Generated: Neutral. Explanation: The sentence does not express any strong emotions or opinions. It is a simple question seeking a suggestion. Therefore, it can be
Target: neutral
Output: neutral
Generated: Neutral. Reasoning: The sentence does not express any strong emotion or opinion. It simply states a fact about Tom and Ray's actions. There is
Target: neutral
Output: neutral
Generated: Neutral. T

In [61]:
print(acc/100)
print(Counter(true_results))
print(Counter(results))

0.76
Counter({'neutral': 69, 'positive': 7})
Counter({'neutral': 79, 'positive': 14, 'unknown': 4, 'negative': 3})


In [1]:
from collections import Counter
def self_consistency(prompt, num_samples=5, max_len=100):
    outputs = []
    for _ in range(num_samples):
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        input_len = inputs["input_ids"].shape[1]
        output = model.generate(
            **inputs,
            max_new_tokens=max_len,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.9,
            num_return_sequences=1,
            pad_token_id = tokenizer.eos_token_id
        )
        generated_ids = output[0][input_len:]
        de = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
        outputs.append(de)
    return outputs
def majority_vote(outputs):
    outputs = [clean_label(output.lower()) for output in outputs]
    counts = Counter(outputs)
    return counts.most_common(1)[0][0], counts

In [76]:
prompt = zero_shot_CoT(data[0]['input'])
print(prompt)
samples = self_consistency(prompt, num_samples=5)
best_answer, all_votes = majority_vote(samples)

print("Self-Consistent Answer:", best_answer)
print("All votes:", all_votes)

### Instruction:
Classify the sentiment of the following sentence into one of the following categories: neutral, positive, or negative.

### Sentence:
between 1900 and 1920 where did most of the migrants to the united states come from

### Response:
Let's think step by step. Sentiment:
Self-Consistent Answer: neutral
All votes: Counter({'neutral': 3, 'positive': 2})


In [77]:
def zero_shot_CoT_SC(text, num_samples=5, max_len=50, do_print=False):
        prompt = zero_shot_CoT(text)
        samples = self_consistency(prompt, num_samples, max_len)
        best_answer, all_votes = majority_vote(samples)
        if do_print:
            print('Self-consistent answer: ', best_answer)
            print('All votes: ', all_votes)
        return best_answer, all_votes

In [79]:
acc = 0
true_results = []
results = []
for i in range(100):
    best_answer, all_votes = zero_shot_CoT_SC(data[i]['input'], do_print=True)
    if clean_label(best_answer.lower()) == data[i]['output']:
        acc = acc + 1
        true_results.append(clean_label(best_answer.lower()))
    results.append(clean_label(best_answer.lower()))
print(acc/100)
print(Counter(true_results))
print(Counter(results))

Self-consistent answer:  neutral
All votes:  Counter({'neutral': 5})
Self-consistent answer:  positive
All votes:  Counter({'positive': 4, 'unknown': 1})
Self-consistent answer:  positive
All votes:  Counter({'positive': 3, 'neutral': 2})
Self-consistent answer:  neutral
All votes:  Counter({'neutral': 4, 'positive': 1})
Self-consistent answer:  positive
All votes:  Counter({'positive': 3, 'unknown': 2})
Self-consistent answer:  neutral
All votes:  Counter({'neutral': 5})
Self-consistent answer:  unknown
All votes:  Counter({'unknown': 3, 'positive': 2})
Self-consistent answer:  positive
All votes:  Counter({'positive': 4, 'negative': 1})
Self-consistent answer:  neutral
All votes:  Counter({'neutral': 4, 'positive': 1})
Self-consistent answer:  positive
All votes:  Counter({'positive': 3, 'neutral': 2})
Self-consistent answer:  neutral
All votes:  Counter({'neutral': 2, 'unknown': 2, 'positive': 1})
Self-consistent answer:  positive
All votes:  Counter({'positive': 4, 'neutral': 1})
S

In [109]:
def few_shots_direct(text):
    return f"""Instruction: Return only the word without prefixes like 'Sentiment:'.
(neutral, positive, or negative).
Sentence: It works as expected. → neutral
Sentence: I love this! → positive
Sentence: This is terrible. → negative
Now classify this sentence
Sentence: {text}
Answer: Sentiment: """


In [110]:
from collections import Counter

acc = 0
results = []
true_results = []

for i in range(100):
    prompt = few_shots_direct(data[i]['input'])
    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
    input_len = inputs['input_ids'].shape[1]

    outputs = generate_output(type=None, input=inputs, max_len=50)

    generated_ids = outputs[0][input_len:]
    decoded = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
    label = clean_label(decoded.lower())

    if label == data[i]['output']:
        acc += 1
        true_results.append(label)

    results.append(label)
    print(decoded)
    print(label)
    print('Target: ', data[i]['output'])

print(f"Accuracy: {acc/100:.2f}")
print("All predictions:", Counter(results))
print("Correct predictions:", Counter(true_results))


Neutral
neutral
Target:  neutral
Positive
positive
Target:  neutral
Neutral
neutral
Target:  positive
Neutral
neutral
Target:  neutral
Positive
positive
Target:  neutral
Neutral
neutral
Target:  positive
Positive
positive
Target:  neutral
Neutral
neutral
Target:  neutral
Neutral
neutral
Target:  neutral
Positive
positive
Target:  neutral
Neutral
neutral
Target:  neutral
Neutral
neutral
Target:  neutral
Neutral
neutral
Target:  neutral
Neutral
neutral
Target:  neutral
Neutral
neutral
Target:  neutral
Neutral
neutral
Target:  neutral
Neutral
neutral
Target:  positive
Neutral
neutral
Target:  negative
Neutral
neutral
Target:  negative
Positive
positive
Target:  neutral
Positive
positive
Target:  neutral
Positive
positive
Target:  neutral
Neutral
neutral
Target:  neutral
Positive
positive
Target:  positive
Positive
positive
Target:  neutral
Neutral
neutral
Target:  neutral
가장 작은 형태소를 찾아보겠습니다.
unknown
Target:  neutral
Neutral
neutral
Target:  neutral
Neutral
neutral
Target:  neutral
Neutral

In [124]:
def few_shots_CoT(text):
    return f"""Instruction: Classify the sentiment of this sentence: {text}
(neutral, positive, or negative).
Here are some examples:
Sentence: It works as expected. → Reason: Neutral tone and wording → Answer: neutral
Sentence: I love this! → Reason: Strong positive words → Answer: positive
Sentence: This is terrible. → Reason: Strong negative tone → Answer: negative

Answer: Sentiment: """
    


In [125]:
acc = 0
results = []
true_results = []
for i in range(100):
  prompt =  few_shots_CoT(data[i]['input'])
  inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
  input_len = inputs['input_ids'].shape[1]
  outputs = generate_output(type=None, input=inputs, max_len=50)
  generate_ids = outputs[0][input_len:]
  de = tokenizer.decode(generate_ids, skip_special_tokens=True)
  if clean_label(de.lower()) == data[i]['output']:
    acc = acc+1
    true_results.append(clean_label(de.lower()))
  results.append(clean_label(de.lower()))
  print(clean_label(de.lower()))
  print('Target: ', data[i]['output'])
print(acc/100)
counts = Counter(results)
print(counts)
counts = Counter(true_results)
print(counts)

neutral
Target:  neutral
neutral
Target:  neutral
neutral
Target:  positive
neutral
Target:  neutral
neutral
Target:  neutral
neutral
Target:  positive
neutral
Target:  neutral
neutral
Target:  neutral
neutral
Target:  neutral
neutral
Target:  neutral
neutral
Target:  neutral
neutral
Target:  neutral
neutral
Target:  neutral
neutral
Target:  neutral
neutral
Target:  neutral
neutral
Target:  neutral
neutral
Target:  positive
neutral
Target:  negative
neutral
Target:  negative
neutral
Target:  neutral
neutral
Target:  neutral
neutral
Target:  neutral
unknown
Target:  neutral
neutral
Target:  positive
neutral
Target:  neutral
neutral
Target:  neutral
neutral
Target:  neutral
neutral
Target:  neutral
neutral
Target:  neutral
positive
Target:  neutral
positive
Target:  neutral
neutral
Target:  neutral
neutral
Target:  neutral
neutral
Target:  neutral
neutral
Target:  neutral
neutral
Target:  neutral
neutral
Target:  positive
neutral
Target:  neutral
neutral
Target:  positive
neutral
Target:

In [131]:
def few_shots_CoT_SC(text, num_samples=5, max_len=100, do_print=False):
        prompt = few_shots_CoT(text)
        samples = self_consistency(prompt, num_samples, max_len)
        best_answer, all_votes = majority_vote(samples)
        if do_print:
            print('Self-consistent answer: ', best_answer)
            print('All votes: ', all_votes)
        return best_answer, all_votes

In [133]:
acc = 0
results = []
true_results = []
for i in range(100):
    best_answer, all_votes = few_shots_CoT_SC(data[i], do_print=True)
    results.append(clean_label(best_answer.lower()))
    if clean_label(best_answer.lower()) == data[i]['output']:
        acc = acc + 1
        true_results.append(clean_label(best_answer.lower()))
    print('Target: ', data[i]['output'])
print(acc/100)
print(Counter(true_results))
print(Counter(results))

Self-consistent answer:  neutral
All votes:  Counter({'neutral': 3, 'positive': 2})
Target:  neutral
Self-consistent answer:  neutral
All votes:  Counter({'neutral': 4, 'positive': 1})
Target:  neutral
Self-consistent answer:  neutral
All votes:  Counter({'neutral': 4, 'positive': 1})
Target:  positive
Self-consistent answer:  neutral
All votes:  Counter({'neutral': 3, 'positive': 2})
Target:  neutral
Self-consistent answer:  neutral
All votes:  Counter({'neutral': 3, 'positive': 2})
Target:  neutral
Self-consistent answer:  neutral
All votes:  Counter({'neutral': 3, 'positive': 2})
Target:  positive
Self-consistent answer:  neutral
All votes:  Counter({'neutral': 4, 'unknown': 1})
Target:  neutral
Self-consistent answer:  positive
All votes:  Counter({'positive': 3, 'neutral': 2})
Target:  neutral
Self-consistent answer:  neutral
All votes:  Counter({'neutral': 5})
Target:  neutral
Self-consistent answer:  neutral
All votes:  Counter({'neutral': 4, 'positive': 1})
Target:  neutral
Sel

In [3]:
import json

with open('/kaggle/input/simple-qa/simple_qa (2).json', 'r', encoding='utf-8') as f:
        qa = json.load(f)

In [146]:
def zero_shot_direct(text):
    return f"""Answer the following question carefully. Show your steps and then provide the final answer.

Question: {text}

Final Answer:"""


In [147]:
input = tokenizer(zero_shot_direct(qa[0]['input']), return_tensors='pt').to('cuda')
input_len = input['input_ids'].shape[1]
output = generate_output(type=None, input=input, max_len = 100)
generate_ids = output[0][input_len:]
print(tokenizer.decode(generate_ids, skip_special_tokens=True))

 The majority of migrants during this time period came from Europe, particularly from countries such as Italy, Ireland, and Germany. These migrants were seeking better economic opportunities and a chance at a better life in the United States. They faced many challenges, including language barriers and discrimination, but their determination and hard work helped shape the country into what it is today. 



In [7]:
from sentence_transformers import SentenceTransformer, util
comp_model = SentenceTransformer('all-MiniLM-L6-V2')
def cosine_similarity_score(pred, target):
    emb1 = comp_model.encode(pred, convert_to_tensor = True)
    emb2 = comp_model.encode(target, convert_to_tensor=True)
    return float(util.cos_sim(emb1, emb2)[0][0])

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [148]:
cosine = 0
for i in range(50):
    input = tokenizer(zero_shot_direct(qa[i]['input']), return_tensors='pt').to('cuda')
    output = generate_output(type=None, input=input, max_len = 300)
    input_len = input['input_ids'].shape[1]
    generate_ids = output[0][input_len:]
    answer = tokenizer.decode(generate_ids, skip_special_tokens=True)
    print('Output: ', answer)
    print('Target: ', qa[i]['output'])
    c = cosine_similarity_score(answer, qa[i]['output'])
    print('Cosine: ', c)
    cosine = cosine + c

Output:   The majority of migrants during this time period came from Europe, particularly from countries such as Italy, Ireland, and Germany. These migrants were seeking better economic opportunities and a chance at a better life in the United States. They faced many challenges, including language barriers and discrimination, but their determination and hard work helped shape the country into what it is today. 

Target:  During the period between 1900 and 1920, most migrants to the United States came from Europe, particularly from Italy, Poland, and Russia. There were also significant numbers of immigrants from Canada, Mexico, and other parts of Latin America. In addition, there were smaller numbers of immigrants from Asia and other parts of the world.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Cosine:  0.7502034902572632
Output:   The meaning of the word "iinola" is not mentioned in the paragraph. However, based on the context, it can be inferred that it is a Yoruba word that means "to be a woman". This is supported by the fact that the name is used for both boys and girls, and is also used as a surname. Therefore, the correct answer is "woman".

Target:  Inioluwa is a Nigerian name that means "God's gift" in Yoruba, a language spoken in Nigeria. It is typically given to a child who is seen as a blessing from God.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Cosine:  0.7566820383071899
Output:   Oak trees grow fastest during the spring season when the weather is warm and there is an abundance of sunlight. During this time, the tree's growth rate is at its peak, allowing it to reach its full potential in terms of height and overall development. This is because the combination of favorable weather conditions and ample sunlight provides the necessary energy for photosynthesis, which is the process by which plants convert sunlight into food. As a result, oak trees are able to absorb more nutrients and grow at a faster rate compared to other seasons. However, it is important to note that growth rates may vary depending on factors such as soil quality, water availability, and genetic factors. Therefore, while spring is generally considered the optimal season for rapid growth, other conditions can also contribute to the overall growth and development of a tree. It is always fascinating to observe the wonders of nature and how different factors ca

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Cosine:  0.8169364929199219


In [149]:
print(cosine/50)

0.6117763109877705


In [5]:
def zero_shot_CoT(text):
        return f"""Answer the following question carefully. Show your steps and then provide the final answer.
                    
                    Question: {text}
                    
                    Final Answer: Let's break it down and then answer clearly, exactly the following."""


In [154]:
cosine = 0
for i in range(50):
    input = tokenizer(zero_shot_CoT(qa[i]['input']), return_tensors='pt').to('cuda')
    output = generate_output(type=None, input=input, max_len = 300)
    input_len = input['input_ids'].shape[1]
    generate_ids = output[0][input_len:]
    answer = tokenizer.decode(generate_ids, skip_special_tokens=True)
    print('Output: ', answer)
    print('Target: ', qa[i]['output'])
    c = cosine_similarity_score(answer, qa[i]['output'])
    print('Cosine: ', c)
    cosine = cosine + c

Output:  

                    Step 1: First, we need to find out the total number of migrants from 1900 to 1920. We can do this by subtracting 1900 from 1920, which gives us 20. This means that there were 20 years of migration during this time period. 

Step 2: Next, let's look at the data for each decade. In the first decade (1900-1909), we have a total of 2.5 million migrants. To find the average for this decade, divide this number by 10 (since there are 10 years in a decade), which equals 250,000 migrants per year. Repeat this process for the other decades, and you will find that the highest average is in the 1910s, with an average of 275,500 migrants/year. Therefore, the answer is that most migrants came from the United States during the decade of 1910-1920, when the migration rate was at its highest.  

Target:  During the period between 1900 and 1920, most migrants to the United States came from Europe, particularly from Italy, Poland, and Russia. There were also significant num

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Cosine:  0.691351592540741
Output:   First, we need to understand the meaning of each part of the word. "In" means "not" or "opposite of". "Io" comes from the Greek word for "eye" and "luwa" is the name of a goddess in the Yoruba religion. So, when we put it all together, it can be translated to mean "the opposite of an eye-goddess". This could refer to something that is not related to or has no connection to the goddess, or it could be used as a negative term to describe someone or something. 

Follow-up Questions:
1. Can you think of any other words that use the prefix "in" to create a new meaning?
2. How does the use of prefixes and suffixes help us understand words better in different languages? 


Topic: Language and Culture

Target:  Inioluwa is a Nigerian name that means "God's gift" in Yoruba, a language spoken in Nigeria. It is typically given to a child who is seen as a blessing from God.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Cosine:  0.769882321357727
Output:  

                    Step 1: Understand the question. The question is asking about the growth rate of oak trees. 
                     Step 2: Analyze the information given. There is no specific information provided about when an Oak tree grows fastest. However, we know that trees generally grow faster in spring and summer due to increased sunlight and warmer temperatures. Therefore, it is reasonable to assume that the answer to this question would be in the spring or summer months, but we can't be certain without more information.  

2. **Exercise:** You are given a set of numbers: 2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 41

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Cosine:  0.7287986278533936


In [16]:
def zero_shot_CoT(text):
        return f"""Answer the following question carefully. Show your steps and then provide the final answer.
                    
                    Question: {text}
                    
                    Final Answer: Let's break it down and then answer clearly, exactly the following. (max 2-3 sentences, do not return irrelevant information, just simply answer)"""


In [18]:
cosine = 0
for i in range(3, 53):
    prompt = zero_shot_CoT(qa[i]['input'])
    samples = self_consistency(prompt, num_samples=5, max_len = 100)
    for sample in samples:
        print('Sample: ', sample)
        print(cosine_similarity_score(sample, qa[i]['output']))
    print('-------------------------------------------------------')

Sample:  Answer: Chicken is a type of poultry that is commonly consumed as food. It is a domesticated bird that is raised for its meat and eggs. Chicken meat is usually white or brown in color and has a mild flavor. It can be cooked in various ways such as grilling, roasting, frying, or baking. Chicken is a popular choice for many dishes and is enjoyed by people all around the world.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.9082370400428772
Sample:  ```python
# Answer to exercise 2

# Here is a simple Python code to calculate the final answer:

chicken = 'what is chicken'

# Step 1: Break down the question and analyze each part

# Step 2: Analyze the word 'what'

# Step 3: Analyze the word 'is'

# Step 4: Analyze the word 'chicken'

# Step 5: Combine the analyzed parts


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.6421726942062378
Sample:  Solution: In Christianity, Jesus addresses deficiencies in God, the Father, by showing his love, mercy, and forgiveness. He demonstrates that God is not perfect, but he is still the source of all good in the world. Jesus also addresses deficiencies in the Father by showing that he is the Son of God, and by being the only way to heaven.

Follow-up Exercise 2:

Question: Explain in detail the concept of God as the source of all good in Christianity


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.6461089253425598
-------------------------------------------------------


In [33]:
def few_shots_direct(text):
        return f"""
            Instruct: Answer the following question, using reliable sources such as wikipedia,...Here are some examples:
            Question:between 1900 and 1920 where did most of the migrants to the united states come from,
            Answer: During the period between 1900 and 1920, most migrants to the United States came from Europe, particularly from Italy, Poland, and Russia. There were also significant numbers of immigrants from Canada, Mexico, and other parts of Latin America. In addition, there were smaller numbers of immigrants from Asia and other parts of the world.

            Question: What does inioluwa mean,
            Answer: Inioluwa is a Nigerian name that means \"God's gift\" in Yoruba, a language spoken in Nigeria. It is typically given to a child who is seen as a blessing from God.

            Question: what is CVD,
            Answer: CVD stands for Cardiovascular Disease. It is a group of conditions that affect the heart and blood vessels. These conditions include coronary artery disease (which can lead to heart attacks),
            
            Now answer clearly, exactly the following: {text}, (max 2-3 sentences)
            Answer:\n 

                """

In [34]:
cosine = 0
for i in range(3,53):
    input = tokenizer(few_shots_direct(qa[i]['input']), return_tensors='pt').to('cuda')
    input_len = input['input_ids'].shape[1]
    output = generate_output(type=None, input = input, max_len = 200)
    generate_ids = output[0][input_len:]
    answer = tokenizer.decode(generate_ids, skip_special_tokens=True)
    print('Answer: ', answer)
    print('Target: ', qa[i]['output'])
    c = cosine_similarity_score(answer, qa[i]['output'])
    print(c)
    cosine = cosine + c

Answer:  Chickens are domesticated birds that are commonly raised for their meat and eggs. They are a popular source of protein and are also used in various cultural and religious practices. Chickens come in a variety of breeds, each with its own unique characteristics and traits.

Target:  Chicken is a type of domesticated bird that is commonly raised for its meat and eggs. It is one of the most widely consumed types of poultry in the world, and is a popular ingredient in many different types of cuisine. Chickens are typically raised on farms, and are fed a diet of grains and other foods to promote their growth and health. Chicken meat is versatile and can be cooked in a variety of ways, such as roasting, grilling, frying, or boiling. Chicken is also a good source of protein and other nutrients, making it a popular choice for many people as part of a healthy diet.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.8784589767456055
Answer:  Eleanor of Aquitaine was born in 1152. She was the daughter of King Louis VII of France and Queen Margaret of Provence. Eleanor was known for her beauty and intelligence and played an important role in the politics of Europe during the Middle Ages.

Target:  Eleanor of Aquitaine was born on either September 24 or 25 in the year 1122. The exact date of her birth is not certain, but it is known that she was born in the city of Bordeaux in southwestern France, which was part of the Duchy of Aquitaine at the time.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.2593420445919037
Answer:  Jesus addresses several deficiencies in God in Christianity. One of these is the deficiency of justice, which is addressed in the Sermon on the Mount. Jesus teaches his followers to love their enemies and to treat others as they would like to be treated. He also teaches them to forgive those who have wronged them, even if they have not been forgiven in return. Another deficiency that Jesus addresses is God's lack of compassion, as seen in his teachings on caring for the sick and the poor. Finally, Jesus also addresses God’s deficiency in mercy, teaching that God is merciful and compassionate towards all people, regardless of their background or circumstances.

Target:  In Christianity, Jesus is seen as the means by which the deficiencies or shortcomings in humanity's relationship with God the Father are addressed. Through his life, death, and resurrection, Jesus is believed to have reconciled humanity to God, making it possible for people to have a direct an

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.6934800148010254


In [35]:
def few_shots_CoT(text):
        return f"""Instruct: Answer the question using reliable information. Think step by step.

                Question:between 1900 and 1920 where did most of the migrants to the united states come from,
                Answer: During the period between 1900 and 1920, most migrants to the United States came from Europe, particularly from Italy, Poland, and Russia. There were also significant numbers of immigrants from Canada, Mexico, and other parts of Latin America. In addition, there were smaller numbers of immigrants from Asia and other parts of the world.

                Question: What does inioluwa mean,
                Answer: Inioluwa is a Nigerian name that means \"God's gift\" in Yoruba, a language spoken in Nigeria. It is typically given to a child who is seen as a blessing from God.

                Question: what is CVD,
                Answer: CVD stands for Cardiovascular Disease. It is a group of conditions that affect the heart and blood vessels. These conditions include coronary artery disease (which can lead to heart attacks),
                
                Now answer clearly, exactly the following: {text}.
                Answer: Let's think step by step before answering. (max 2-3 sentences)"""

In [36]:
print(cosine/50)

0.5745592728070915


In [38]:
cosine = 0
for i in range(3,53):
    input = tokenizer(few_shots_CoT(qa[i]['input']), return_tensors='pt').to('cuda')
    input_len = input['input_ids'].shape[1]
    output = generate_output(type=None, input = input, max_len = 100)
    generate_ids = output[0][input_len:]
    answer = tokenizer.decode(generate_ids, skip_special_tokens=True)
    print('Answer: ', answer)
    print('Target: ', qa[i]['output'])
    c = cosine_similarity_score(answer, qa[i]['output'])
    print(c)
    cosine = cosine + c

Answer:  

Exercises:
1. What is the difference between a primary source and a secondary source?
2. Why is it important to evaluate the credibility of a source before using it in a research project? 
3. How can you tell if a website is reliable or not? Give at least three examples. 


Real-world use case 1: Researching a school project
Samantha and her classmates are working on a project about the history of their town.
Target:  Chicken is a type of domesticated bird that is commonly raised for its meat and eggs. It is one of the most widely consumed types of poultry in the world, and is a popular ingredient in many different types of cuisine. Chickens are typically raised on farms, and are fed a diet of grains and other foods to promote their growth and health. Chicken meat is versatile and can be cooked in a variety of ways, such as roasting, grilling, frying, or boiling. Chicken is also a good source of protein and other nutrients, making it a popular choice for many people as part 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.059389226138591766
Answer:   Eleanor of Aquitaine was born in 1152. She was the daughter of King Henry II of England and Queen Matilda of Boulogne. After her father's death, she became the queen of France and played a significant role in the Hundred Years' War. 

Exercises:
1. What is the difference between a fact and an opinion? Answer: A fact is something that can be proven to be true or false, based on evidence or data. An opinion
Target:  Eleanor of Aquitaine was born on either September 24 or 25 in the year 1122. The exact date of her birth is not certain, but it is known that she was born in the city of Bordeaux in southwestern France, which was part of the Duchy of Aquitaine at the time.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.382606565952301
Answer:  

1. Deficiencies in God can be addressed in Christianity through the teachings of Jesus Christ. Jesus is believed to be the Son of God and the Messiah, who came to Earth to save humanity from sin and death. He taught his followers to love God with all their heart, soul, mind and strength, as well as love their neighbors as themselves. By following Jesus' teachings, Christians believe that they can overcome their deficiencies and live a life of holiness and righteousness. 
2
Target:  In Christianity, Jesus is seen as the means by which the deficiencies or shortcomings in humanity's relationship with God the Father are addressed. Through his life, death, and resurrection, Jesus is believed to have reconciled humanity to God, making it possible for people to have a direct and personal relationship with Him. This reconciliation is seen as overcoming the estrangement caused by sin and restoring the original relationship between God and humanity as it was intended

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.6604933738708496


In [40]:
print(cosine/50)

0.40654573960229756


In [41]:
cosine = 0
for i in range(3, 53):
    prompt = few_shots_CoT(qa[i]['input'])
    samples = self_consistency(prompt, num_samples=5, max_len = 100)
    for sample in samples:
        print('Sample: ', sample)
        print(cosine_similarity_score(sample, qa[i]['output']))
    print('-------------------------------------------------------')

Sample:  Chicken is a type of bird that is raised for its meat. It is one of the most popular meats in the world and is consumed in many different forms, including roasted, fried, grilled, and in soups and stews.

                Question: what is the capital of France,
                Answer: The capital of France is Paris. Paris is a city in northern France that is known for its art, culture, and architecture. It is home to many famous landmarks, including the Eiff


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.6482501029968262
Sample:  Chicken is a type of domesticated bird that is commonly raised for its meat and eggs. It is a popular source of protein and is consumed all over the world.

4. You are at a family gathering and your grandparents start talking about their childhood. They mention that they used to have a pet chicken that they would take care of. You want to ask them more about it, but you're not sure how to phrase your question.

Instruct: Answer the question


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.6846842765808105
Sample:  In Christianity, Jesus is believed to have been sent by God to address the deficiencies or shortcomings that humans have. He is seen as the savior who can remove the deficiencies of humanity and bring about redemption and salvation. Jesus is believed to be the embodiment of God's love and compassion, and he is seen as the ultimate source of hope and forgiveness.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.7972848415374756
-------------------------------------------------------


# Advanced prompt techniques